In [5]:
import sys,os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # 다양한 차트를 그려보기 위해서 
import datetime as dt # 시간을 나타내기 위해서
import json # JSON 데이터 처리하기 위해서
import folium # 지도를 그려보기 위해서 
from folium.plugins import MarkerCluster
plt.rcParams['font.family']='Malgun Gothic' # 차트에서 한글 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False  # 차트에서 -기호 깨짐 방지

sys.path.append('c:/test')
os.chdir('c:/test')

In [8]:
def create_apt(filename):
    apt=pd.read_excel(filename)
    apt.columns=apt.iloc[0]
    apt.columns=['연번','단지명','용도','구청','행정동','주소','대지면적','연면적','건폐율','용적율',
                 '동수','층수','세대수','60이하','60~85','85초과','사업승인','사용검사','사업주체','시공자']
    apt.drop([0,1],inplace=True) # .drop([여기에 버리는 행을 리스트의 요소로 써야함!!])
    apt.set_index('연번',inplace=True) # 연번 칼럼의 value를 인덱스로 설정
    apt.fillna(0,inplace=True) # value값에 대해서 NAN 데이터 정수 0으로 처리해준다.
    apt['사용년도']=dt.date.today().year - pd.to_datetime([str(i) for i in apt['사용검사']]).year
    apt=apt[['단지명', '행정동', '주소', 
           '층수', '세대수', '60이하', '60~85', '85초과', '사업승인', '사용검사',  '사용년도']]
    return apt


In [10]:
# apt df 확인
apt = create_apt('공동주택현황_2023_3.xls')
apt.head()

,단지명,행정동,주소,층수,세대수,60이하,60~85,85초과,사업승인,사용검사,사용년도
연번,,,,,,,,,,,
1,동양맨션,진북동,진북동 417-47,5,180,0,90,90,1977-11-26 00:00:00,1978-07-05,45
2,삼우,서서학동,서서학동 372-50,5,120,0,120,0,1977-12-15 00:00:00,1978-11-02,45
3,기린(인후),인후1동,인후1가 530-1,5,100,100,0,0,1978-03-06 00:00:00,1978-11-02,45
4,거북연립,금암1동,금암동 752-3,4,32,1,31,0,1978-04-25 00:00:00,1978-11-07,45
5,동화연립,금암2동,금암2동 525-7,4,38,0,22,16,1979-06-20 00:00:00,1980-03-27,43


In [13]:

def create_local_df(start,end,locals): # 행정동()별 세대수 총합 구해보기
    r_apt=apt[(apt['사용년도']>=start) & (apt['사용년도']<=end)]
    temp=[]
    for local in locals:
        s=r_apt[r_apt['행정동'].str.contains(local)]['세대수'].sum()
        temp.append(s)
    local_df=pd.DataFrame(temp,index=locals)

    return local_df

local_df = create_local_df(20,30,['송천','평화'])
local_df

,0
송천,9092
평화,12954


In [14]:
def create_dong_df(start,end,locals): # 행정동별 세대수 총합 구해보기
    r_apt=apt[(apt['사용년도']>=start) & (apt['사용년도']<=end)]
    dongs_df= pd.DataFrame()
    for local in locals:
        dong=r_apt[r_apt['행정동'].str.contains(local)].groupby('행정동')['세대수'].sum()
        dongs_df=pd.concat([dongs_df,dong])
#     dongs_df.reset_index(inplace=True)
#     dongs_df.columns=['동','세대수']
    return dongs_df

dongs_df = create_dong_df(20,30,['송천','평화'])
dongs_df

,0
송천1동,4207.0
송천2동,4885.0
평화1동,1300.0
평화2동,11654.0


In [16]:
start=20
end=30
locals=['효자','평화','송천','인후']
idx=locals
locals_df=pd.DataFrame(index=idx)
for i in range(8):
    df=create_local_df(start-i,end-i,locals)
  
    col_name=str(dt.date.today().year+i) + '년'
    df.columns=[col_name]

    locals_df=pd.merge(locals_df, df[col_name], how='left', left_index= True, right_index= True)
    
locals_df.dropna(how='all',inplace=True) # 모든 열의 값이 NA면 해당 행 버려라
locals_df.fillna(0,inplace=True) # 모든 열의 값은 아니지만 하나 이상의 값이 NA면 그 부분은 0으로 채우라는 의미
locals_df.sort_index(inplace=True) # 인덱스 기준 정렬

In [26]:
locals_df.loc['평균',:]=locals_df.mean() # 기본 axis=0 세로로 계산해줌 기본값이다 <인덱스에 추가>
locals_df.loc[:,'평균']=locals_df.mean(axis=1) # 기본 axis=1 가로로 계산해줌 <칼럼에 추가>
# locals_df['평균'] = locals_df.mean(axis=1) <칼럼에 추가>

# locals_df['평균'].drop(inplace=True)
# locals_df.loc[:,'평균'].drop(inplace=True)
# del(locals_df['평균'])
locals_df.drop(['평균'])
locals_df

,2023년,2024년,2025년,2026년,2027년,2028년,2029년,2030년,평균
송천,9092.00,8062.00,7427.0,7573.0,7156.0,8025.0,5332.00,4555.0,7152.75000
인후,8098.00,8319.00,8319.0,9653.0,11174.0,11361.0,9338.00,5251.0,8939.12500
평화,12954.00,12944.00,14309.0,11489.0,9851.0,9363.0,7805.00,7332.0,10755.87500
효자,8213.00,7354.00,7161.0,7931.0,11869.0,10947.0,11298.00,11780.0,9569.12500
평균,9589.25,9169.75,9304.0,9161.5,10012.5,9924.0,8443.25,7229.5,9104.21875


In [25]:
x = dongs_df.columns
col='rbkyg'*(100//5+1)
print(col)

rbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkygrbkyg
